In [1]:
from database.db_config import current_db as db

In [2]:
# # Alter the headers types, before querying.
# db.execute_ddl('ALTER TABLE "All continents updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer, ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')
# db.execute_ddl('ALTER TABLE "All countries updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer,ALTER COLUMN "scrap_date" SET DATA TYPE date USING to_date(cast("scrap_date" as TEXT), \'yyyy-mm-dd\')')


In [3]:
countries_minMax = db.sql_query('SELECT "Country", "TotalCases" FROM "All countries updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All countries updated" LIMIT 1),'
						                       '((SELECT mAX("TotalCases") FROM "All countries updated"LIMIT 1)))')
print(countries_minMax)

+---------+------------+
| Country | TotalCases |
+---------+------------+
|  India  |  9884716   |
| Vanuatu |     1      |
+---------+------------+


In [4]:
# TotalCases Min and Max by a continent.
continents_minMax = db.sql_query('SELECT "Continent", "TotalCases" FROM "All continents updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All continents updated" WHERE "Continent" <>  \'World\' LIMIT 1),'
						                       '((SELECT mAX("TotalCases") FROM "All continents updated" WHERE "Continent" <>  \'World\'LIMIT 1)))')
print(continents_minMax)

+-----------+------------+
| Continent | TotalCases |
+-----------+------------+
|   Europe  |  19828718  |
|  Oceania  |   46571    |
+-----------+------------+


In [5]:
# Top 15 countries with the highest number of cases.
top15_totalCases = db.sql_query('SELECT "Country","Continent","TotalCases",RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "Continent_rank_by_TotalCases","ActiveCases","SeriousCritical" FROM (SELECT "Country_id","Continent_id" FROM "All Countries") AS t1 JOIN  (SELECT * FROM "All Continents") AS t2 USING("Continent_id") JOIN  (SELECT "Country","Country_id","TotalCases","ActiveCases","SeriousCritical"  FROM "All countries updated" LIMIT 15) AS t3 USING("Country_id")')
print(top15_totalCases)

+-----------+---------------+------------+------------------------------+-------------+-----------------+
|  Country  |   Continent   | TotalCases | Continent_rank_by_TotalCases | ActiveCases | SeriousCritical |
+-----------+---------------+------------+------------------------------+-------------+-----------------+
|   India   |      Asia     |  9884716   |              1               |    353715   |       8944      |
|   Turkey  |      Asia     |  1836728   |              2               |    216531   |       5973      |
|    Iran   |      Asia     |  1108269   |              3               |    243803   |       5723      |
|   Russia  |     Europe    |  2653928   |              1               |    500752   |       2300      |
|   France  |     Europe    |  2376852   |              2               |   2141946   |       2871      |
|     UK    |     Europe    |  1849403   |              3               |     None    |       1275      |
|   Italy   |     Europe    |  1843712   |    

In [6]:
# Countries that are located in Oceania continent.
oceania_countries = db.sql_query('SELECT "Country" FROM "All Countries" WHERE "Continent_id" IN (SELECT '
                                 '"Continent_id" FROM "All Continents" WHERE "Continent" = \'Oceania\')')
print(oceania_countries)

+------------------+
|     Country      |
+------------------+
|    Australia     |
|   New Zealand    |
| French Polynesia |
| Papua New Guinea |
|       Fiji       |
|  New Caledonia   |
+------------------+


In [7]:
# The count of countries in each continent.
countries_perContinent = db.sql_query('SELECT "Continent","Countires_count" FROM (SELECT * FROM "All Continents") AS ''t1 JOIN (SELECT "Continent_id", COUNT("Country") AS "Countires_count" FROM "All Countries" GROUP BY "Continent_id") AS t2 USING("Continent_id") ORDER BY "Countires_count" DESC')
print(countries_perContinent)

+---------------+-----------------+
|   Continent   | Countires_count |
+---------------+-----------------+
|     Africa    |        57       |
|      Asia     |        49       |
|     Europe    |        48       |
| North America |        39       |
| South America |        14       |
|    Oceania    |        6        |
+---------------+-----------------+


In [8]:
# The country with the highest "TotalCases" in each continent.
country_high_perContinent = db.sql_query('SELECT "Country","Continent","TotalCases" FROM (SELECT "Country","Continent","TotalCases" ,RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "rank" FROM (SELECT "Country","Country_id","TotalCases" FROM "All countries updated" ) AS t1  JOIN (SELECT "Country_id", "Continent_id" FROM "All Countries") AS t2 USING("Country_id")  JOIN (SELECT "Continent_id","Continent" FROM "All Continents") AS t3 USING("Continent_id")) AS t4 WHERE "rank" =1;')
print(country_high_perContinent)

+------------------------+---------------+------------+
|        Country         |   Continent   | TotalCases |
+------------------------+---------------+------------+
|         Gabon          |     Africa    |    9330    |
|         India          |      Asia     |  9884716   |
|        Slovenia        |     Europe    |   96314    |
| St. Vincent Grenadines | North America |     98     |
|    Papua New Guinea    |    Oceania    |    720     |
|          Peru          | South America |   983045   |
+------------------------+---------------+------------+


In [9]:
# Top 5 countries with active cases.
top5_countries_active = db.sql_query('SELECT "Country","ActiveCases" FROM "All countries updated" WHERE "ActiveCases" '
                                   'IS NOT NULL ORDER BY "ActiveCases" DESC LIMIT 5;')
print(top5_countries_active)

+--------------+-------------+
|   Country    | ActiveCases |
+--------------+-------------+
| Burkina Faso |     965     |
|  Luxembourg  |     9638    |
|    Congo     |     962     |
|    Egypt     |     9481    |
|    Yemen     |      94     |
+--------------+-------------+


In [10]:
# The date that israel had the highest value of new cases.
israel_newCases_date = db.sql_query('SELECT "scrap_date" AS "max_NewCases_date" FROM "Israel" WHERE "NewCases" = (Select MAX("NewCases") FROM "Israel")')
print(israel_newCases_date)

+-------------------+
| max_NewCases_date |
+-------------------+
|     2020-09-27    |
+-------------------+


In [11]:
# Ranking each continent by its active cases.
continent_active_rank = db.sql_query('SELECT "Continent","ActiveCases", RANK() OVER(ORDER BY "ActiveCases" DESC) FROM "All continents updated" WHERE "Continent" <> \'World\'')
print(continent_active_rank)

+---------------+-------------+------+
|   Continent   | ActiveCases | rank |
+---------------+-------------+------+
| North America |   7148880   |  1   |
|     Africa    |    312095   |  2   |
|      Asia     |   1506076   |  3   |
|    Oceania    |    12324    |  4   |
| South America |   1027649   |  5   |
|     Europe    |   10120134  |  6   |
+---------------+-------------+------+


In [12]:
israel_minMax_dates = db.sql_query('SELECT MIN("scrap_date") as "Earliest Date",MAX("scrap_date") as "Latest Date" FROM "Israel"')
print(israel_minMax_dates)

+---------------+-------------+
| Earliest Date | Latest Date |
+---------------+-------------+
|   2020-08-02  |  2020-12-13 |
+---------------+-------------+
